In [1]:
# import pandas as pd 
# import numpy as np
# import csv
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import r2_score

# ##################################### DATA LOADING ##################################
# data = pd.read_csv('./Data/EURUSD_D1.csv')
# data['Time'] = pd.to_datetime(data['Time'],format='%Y-%m-%d %H:%M:%S')
# data.set_index('Time', inplace=True)

# #####################################################################################


# #Linear regression Model
# class LinearRegression:
#     def __init__(self, learning_rate=0.01, n_iterations=1000):
#         self.learning_rate = learning_rate
#         self.n_iterations = n_iterations
#         self.weights = None
#         self.bias = None

#     def fit(self, X, y):
#         num_samples, num_features = X.shape
#         self.weights = np.zeros(num_features)
#         self.bias = 0

#         for _ in range(self.n_iterations):
#             # Calculate predictions
#             y_pred = np.dot(X, self.weights) + self.bias

#             # Compute gradients
#             dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
#             db = (1 / num_samples) * np.sum(y_pred - y)

#             # Update weights and bias
#             self.weights -= self.learning_rate * dw
#             self.bias -= self.learning_rate * db

#     def predict(self, X):
#         return np.dot(X, self.weights) + self.bias

#Lag feature Engineering (X + Y)
# def featureEngineering(data, target_column,  n_past=1):
#     X = []
#     Y= []
#     for i in range(n_past, len(data)):
#         #Append dates
#         X.append(data.iloc[i-n_past:i, target_column].values)  # Select n_past number of rows for the closing price
#         Y.append(data.iloc[i, target_column])  # The target value

#     feature_columns = [f'n_{i}' for i in range(1, n_past+1)]
#     X_df = pd.DataFrame(X, columns=feature_columns)
#     Y_df = pd.DataFrame(Y, columns=['response'])

#     data = pd.concat([X_df, Y_df], axis=1)

#     return data, X_df,Y_df

# ####################################################### ML Model Training #################################

# #data processing
# lookback_window = 5
# learning_rate = 0.01
# target_column=-2

# full_dataset, X, Y =  featureEngineering(data, target_column, lookback_window)
# print(X.head())

# #train test split
# X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=10, shuffle=False)

# print(X_test.head())
# # Reshape y_train and y_test from DataFrame to 1D array or Series
# Y_train = Y_train.values.ravel()
# Y_test = Y_test.values.ravel()    

# # Data standardization/Normalizstion
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Model fit
# print({Y_train.shape})
# LR_model =  LinearRegression(learning_rate)
# LR_model.fit(X_train,Y_train)

# # Model prediction
# y_predict = LR_model.predict(X_test)

# test_mae =  mean_absolute_error(Y_test, y_predict)
# print(f'MAE: {test_mae}')

# ###################################### Strategy and backtest #################################################

# from backtesting import Backtest, Strategy

# #Strategy if prediction high buy, low sell

# class forecastStrat(Strategy):

#     def init(self):
#         self.mlr = LinearRegression()
#         self.mlr.fit(X_train, Y_train)
        
#         #forecast
#         self.future = self.mlr.predict(X_test)

#         '''
#         NOTE: exclude training data?
#         '''
    
#     #Simulate market execute
#     def next(self):

#         # if forcasted value higher: buy
#         if(self.future[-1] > self.data.X_test[-1]):
#             self.position.close() # close open postionn
#             self.buy() # buy 

#         # else if forecast low: sel
#         elif (self.future[-1]< self.data.X_test[-1] ):
#             self.position.close()
#             self.sell()

# bt = Backtest(X_test, forecastStrat, cash=10000, commission=0.001)
# stats =  bt.run()
# stats


# # from backtesting.test import GOOG

# # GOOG.tail()


# # import pandas as pd

# # def SMA(values, n):
# #     """
# #     Return simple moving average of `values`, at
# #     each step taking into account `n` previous values.
# #     """
# #     return pd.Series(values).rolling(n).mean()


# # from backtesting import Strategy
# # from backtesting.lib import crossover
# # from backtesting import Backtest

# # class SmaCross(Strategy):
# #     # Define the two MA lags as *class variables*
# #     # for later optimization
# #     n1 = 10
# #     n2 = 20
    
# #     def init(self):
# #         # Precompute the two moving averages
# #         self.sma1 = self.I(SMA, self.data.Close, self.n1)
# #         self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
# #     def next(self):
# #         # If sma1 crosses above sma2, close any existing
# #         # short trades, and buy the asset
# #         if crossover(self.sma1, self.sma2):
# #             self.position.close()
# #             self.buy()

# #         # Else, if sma1 crosses below sma2, close any existing
# #         # long trades, and sell the asset
# #         elif crossover(self.sma2, self.sma1):
# #             self.position.close()
# #             self.sell()




# # bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
# # stats = bt.run()
# # stats

# # bt.plot()



import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime

# Load data
data = pd.read_csv('./Data/EURUSD_D1.csv')
data['Time'] = pd.to_datetime(data['Time'], format='%Y-%m-%d %H:%M:%S')
data.set_index('Time', inplace=True)

# Linear Regression Model
class LinearRegressionModel:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            y_pred = np.dot(X, self.weights) + self.bias
            dw = (1 / num_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / num_samples) * np.sum(y_pred - y)
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# Feature engineering for lag
def featureEngineering(data, target_column, n_past=1):
    X, Y = [], []
    for i in range(n_past, len(data)):
        X.append(data.iloc[i-n_past:i, target_column].values)
        Y.append(data.iloc[i, target_column])
    return np.array(X), np.array(Y)

# Data Preparation
target_column = -2  # Assuming Close price is the target
n_past = 5  # Lookback window size
X, Y = featureEngineering(data, target_column, n_past)

    
# Train/Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=10, shuffle=False)

Train_Size = len(X_train)   

# Scaling the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model
model_lr = LinearRegressionModel(learning_rate=0.01, n_iterations=1000)
model_lr.fit(X_train, Y_train)


# Backtesting Strategy using Linear Regression
class LinearRegressionStrategy(Strategy):
    def init(self):
        # self.model = LinearRegressionModel(learning_rate=0.01, n_iterations=1000)

        # Data Preparation
        # target_column = -2  # Assuming Close price is the target
        # n_past = 5  # Lookback window size
        # self.X, self.Y = featureEngineering(self.data.df, target_column, n_past)

        
        # Train/Test Split
        # X_train, X_test, Y_train, Y_test = train_test_split(self.X, self.Y, test_size=0.2, random_state=10, shuffle=False)

        # self.Train_Size = len(X_train)   


        # Scaling the data
        # scaler = StandardScaler()
        # X_train = scaler.fit_transform(X_train)
        # X_test = scaler.transform(X_test)

        # Train the model
        # self.model.fit(X_train, Y_train)
        
        # Make predictions on the test data
        self.model= model_lr
        self.predicted_prices = self.model.predict(X_test)
        self.Actual_price = Y_test

        #Plot foreccasted results
       
        i =0
        if i ==0:
            i= i+1
        else:
            i= i+1
        

        print(f'Init iteration :{i}')

        # Track positions and predictions
        self.index = 0

    def next(self):

        # skip training/ in - sample data
        # if len(self.data) < self.Train_Size:
        #     # print(len(self.data))
        #     return
        

        # We move through test set predicand not self.position.is_long tions
        if self.index < len(self.predicted_prices) - 1:
            pred_price_today = self.predicted_prices[self.index]
            pred_price_tomorrow = self.Actual_price[self.index]

            # Buy if tomorrow's predicted price is higher than today's predicted price
            if pred_price_tomorrow > pred_price_today and not self.position.is_long:
                self.position.close()
                self.buy()
            # Sell if tomorrow's predicted price is lower than today's predicted price
            elif pred_price_tomorrow < pred_price_today and not self.position.is_short:
                self.position.close()
                self.sell()

            self.index += 1
        
        # print(f'Indx value: {self.index}')

# Set up Backtest and run
test_data = data.iloc[-len(X_test):]  # This assumes X_test is at the end of the dataset
print(test_data.head())
bt = Backtest(test_data, LinearRegressionStrategy, cash =10000, commission=.002, margin=.05, trade_on_close=False)
stats = bt.run()
bt.plot()

stats



/home/dgibrilly/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


               Open     High      Low    Close  Volume
Time                                                  
2021-05-28  1.21927  1.22048  1.21325  1.21905   95930
2021-05-30  1.21990  1.21998  1.21890  1.21892    2698
2021-05-31  1.21893  1.22343  1.21832  1.22325   58587
2021-06-01  1.22325  1.22542  1.22110  1.22174   81752
2021-06-02  1.22172  1.22265  1.21640  1.22094   77805
Init iteration :1


Start                     2021-05-28 00:00:00
End                       2024-08-09 00:00:00
Duration                   1169 days 00:00:00
Exposure Time [%]                   99.800399
Equity Final [$]                     0.208476
Equity Peak [$]                       10000.0
Return [%]                         -99.997915
Buy & Hold Return [%]               -10.44912
Return (Ann.) [%]                  -93.350995
Volatility (Ann.) [%]               15.180913
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.997915
Avg. Drawdown [%]                  -99.997915
Max. Drawdown Duration     1167 days 00:00:00
Avg. Drawdown Duration     1167 days 00:00:00
# Trades                                  232
Win Rate [%]                         28.87931
Best Trade [%]                         3.8434
Worst Trade [%]                     -2.037099
Avg. Trade [%]                    